In [1]:
import pandas as pd
import scanpy.api as sc
import anndata as 
import scipy.io as sio
import numpy as np
import scipy.sparse
import scipy
import seaborn as sb
from sklearn.preprocessing import normalize
from pylab import *
fsize=14
%matplotlib inline

/net/gs/vol3/software/modules-sw/python/3.5.2/Linux/RHEL6/x86_64/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Path change required (you have the "pipeline" folder on hard drive)

barcodes_8nt = pd.read_csv('/net/shendure/vol8/projects/BCBL/Mixing/ref/barcodes_96.csv',names=['barcodes'],index_col=0).barcodes
barcode2well = dict(zip(barcodes_8nt.values,list(range(48))+list(range(48))))

barcodes_8nt_v2 = pd.read_csv('/net/shendure/vol8/projects/BCBL/Mixing/ref/barcodes_8nt_v2.csv',names=['barcodes']).barcodes
barcode2well_v2 = dict(zip(barcodes_8nt_v2.values,list(range(48))+list(range(48))))



def load_data_combine_hex_combine_genes(filenames,read_cutoff=100):
    if type(filenames)==str:
        df = pd.read_csv(filenames,names=['barcode','gene'])
        df['gene'] = df.gene.apply(lambda s:s.split('_')[0])
    else:
        dfs = []
        for f in filenames:
            print (f),
            cur_df = pd.read_csv(f,names=['barcode','gene'])
            cur_df['gene'] = cur_df.gene.apply(lambda s:s.split('_')[0]+'_'+s.split('_')[-1])
            cur_df['barcode'] = cur_df.barcode.str.slice(0,24)+'_'+f.split('/')[-2]
            dfs.append(cur_df)
        df = pd.concat(dfs)
    df['barcode'] = df.barcode.str.slice(0,16)+'_'+df.barcode.apply(lambda s:str(barcode2well_v2[s[16:24]])) + df.barcode.str.slice(24)
    reads_per_cell = df.groupby(df.barcode).size()
    cells = reads_per_cell[reads_per_cell>3]
    all_genes = df.gene.unique()
    gene_dict = dict(zip(all_genes,range(len(all_genes))))
    cell_dict = dict(zip(cells.index.values,range(len(cells.index.values))))
    rows,cols,vals = [],[],[]
    for bc,g in zip(df.barcode.values,df.gene.values):
        try:
            cell_dict[bc]
        except:
            pass
        else:
            rows.append(cell_dict[bc])
            cols.append(gene_dict[g])
            vals.append(1)
    rows.append(len(cell_dict)-1)
    cols.append(len(gene_dict)-1)
    vals.append(0)
    digital_count_matrix = scipy.sparse.csr_matrix((vals,(rows,cols)),dtype=np.float64)
    thresholded_cells = np.array(digital_count_matrix.sum(1)).flatten()>read_cutoff
    digital_count_matrix = digital_count_matrix[thresholded_cells,:]
    expressed_genes = np.array(digital_count_matrix.sum(0)).flatten()>0
    all_genes = pd.Series(all_genes[expressed_genes])
    digital_count_matrix = digital_count_matrix[:,expressed_genes]
    barcodes = cells.index.values[thresholded_cells]
    return digital_count_matrix,all_genes,barcodes

def barnyard(digital_count_matrix,all_genes,tickstep=2500,s=4,lim=None):
    colors = list(sb.color_palette('Set1',n_colors=2)) + ['gray']
    sb.set_style("white")
    sb.set_style("ticks")
    fig = figure(figsize=(3,3))
    ax = fig.add_subplot(111)
    human_genes = all_genes.str.slice(-5)=='HUMAN'
    mouse_genes = all_genes.str.slice(-5)=='MOUSE'
    human_counts = np.array(digital_count_matrix[:,find(human_genes)].sum(1)).flatten()
    mouse_counts = np.array(digital_count_matrix[:,find(mouse_genes)].sum(1)).flatten()
    human_cells = human_counts>(mouse_counts*9)
    mouse_cells = mouse_counts>(human_counts*9)
    mixed_cells = ~(human_cells|mouse_cells)
    scatter(human_counts[mixed_cells],
            mouse_counts[mixed_cells],
            color=colors[2],
            s=s)
    scatter(human_counts[mouse_cells],
            mouse_counts[mouse_cells],
            color=colors[0],
            s=s,
            alpha=1,
           )
    scatter(human_counts[human_cells],
            mouse_counts[human_cells],
            color=colors[1],
            s=s)

    scatter([],[],
            color=colors[0],
            s=10,
            label='%d Mouse (%0.1f'%(sum(mouse_cells),100*float(sum(mouse_cells))/len(mouse_cells))+'%)',
           )
    scatter([],[],
            color=colors[1],
            label='%d Human (%0.1f'%(sum(human_cells),100*float(sum(human_cells))/len(human_cells))+'%)',
            s=10)
    scatter([],[],
            color=colors[2],
            label='%d Mixed (%0.1f'%(sum(mixed_cells),100*float(sum(mixed_cells))/len(mixed_cells))+'%)',
            s=10)
    if lim==None:
        lim = int(digital_count_matrix.sum(1).max()*1.1)
    ax.set_xticks(arange(0,lim,tickstep))
    ax.set_yticks(arange(0,lim,tickstep))
    ax.set_xticklabels(arange(0,lim,tickstep),rotation=90)
    ax.axis([-int(lim/30.),lim,-int(lim/30.),lim])
    ax.set_xlabel('Human UMI Counts',fontsize=fsize)
    ax.set_ylabel('Mouse UMI Counts',fontsize=fsize)
    ax.tick_params(labelsize=fsize)
    ax.yaxis.tick_left()
    ax.xaxis.tick_bottom()
    ax.legend(bbox_to_anchor=(1.01,1.0),fontsize=fsize-1,handletextpad=0.025)
    #fig.savefig(filename+'.png',dpi=300,bbox_inches='tight')
    #fig.savefig(filename+'.pdf',dpi=300,bbox_inches='tight')
    return fig,ax

def get_stats(digital_count_matrix,all_genes):
    human_genes = all_genes.str.slice(-5)=='HUMAN'
    mouse_genes = all_genes.str.slice(-5)=='MOUSE'
    human_counts = np.array(digital_count_matrix[:,find(human_genes)].sum(1)).flatten()
    mouse_counts = np.array(digital_count_matrix[:,find(mouse_genes)].sum(1)).flatten()
    human_cells = human_counts>(mouse_counts*9)
    mouse_cells = mouse_counts>(human_counts*9)
    human_gene_counts = []
    human_umi_counts = np.array(digital_count_matrix[find(human_cells),:].sum(1)).flatten()
    mouse_umi_counts = np.array(digital_count_matrix[find(mouse_cells),:].sum(1)).flatten()
    for ind in find(human_cells):
        human_gene_counts.append(len(pd.Series(all_genes)[find(digital_count_matrix[ind,:].todense()>0)].apply(lambda s:s.split('_')[0]).unique()))
    mouse_gene_counts = []
    for ind in find(mouse_cells):
        mouse_gene_counts.append(len(pd.Series(all_genes)[find(digital_count_matrix[ind,:].todense()>0)].apply(lambda s:s.split('_')[0]).unique()))
    print( 'Mean number of genes in human cells:', mean(human_gene_counts))
    print( 'Mean number of genes in mouse cells:', mean(mouse_gene_counts))
    print( 'Median number of genes in human cells:', median(human_gene_counts))
    print( 'Median number of genes in mouse cells:', median(mouse_gene_counts))
    print( 'Mean number of UMIs in human cells:', mean(human_umi_counts))
    print( 'Mean number of UMIs in mouse cells:', mean(mouse_umi_counts))
    print( 'Median number of UMIs in human cells:', median(human_umi_counts))
    print( 'Median number of UMIs in mouse cells:', median(mouse_umi_counts))
    print( 'Mean human purity',mean(human_counts[human_cells]/(human_counts[human_cells]+mouse_counts[human_cells])))
    print( 'Mean mouse purity',mean(mouse_counts[mouse_cells]/(human_counts[mouse_cells]+mouse_counts[mouse_cells])))
    print( 'Fraction human cells', mean(human_cells))
    print( 'Fraction mouse cells', mean(mouse_cells))
    print( 'Fraction multiplet cells', 1-mean(mouse_cells|human_cells))
    print( 'Total Cells', len(human_cells))
    return human_gene_counts,mouse_gene_counts

def print_dup_rate(filedir):
    filtered_barcodes_TAG = pd.read_csv(filedir+'read_barcodes_filtered.txt',names=['barcodes']).barcodes.values
    filter_barcode_dict_TAG  = dict(zip(list(filtered_barcodes_TAG ),np.ones(len(filtered_barcodes_TAG ))))
    samfile = pysam.Samfile(filedir+'/star_gene_tagged.bam')
    barcode_umis = []
    seqs = []
    genes = []
    c = 0
    for read in samfile:
        cur_bc = read.qname[:35]
        cur_bc = cur_bc[:24]+cur_bc[25:35]
        try:
            filter_barcode_dict_TAG[cur_bc]
            if not read.is_secondary:
                barcode_umis.append(read.qname[:35])
                seqs.append(read.seq)
                try:
                    genes.append(dict(read.tags)['GE'])
                except:
                    genes.append('')
        except:
            pass
        if c %100000==0:
            print(c,end=' ')
        c+=1
        if c>3000000:
            break
    barcode_umis = pd.Series(barcode_umis)
    seqs = pd.Series(seqs)
    genes = pd.Series(genes)
    samfile.close()
    barcode_umi_counts = barcode_umis.groupby(barcode_umis).apply(size)
    print()
    print('Dup rate:',1 - sum(barcode_umi_counts>0)/float(sum(barcode_umi_counts)))
    print()

In [3]:
def plot_read_thresh(sub_inds=None):
    if sub_inds is None:
        sub_inds = arange(len(barcodes))
    read_counts = pd.Series(index=barcodes[sub_inds],data=np.array(digital_count_matrix[sub_inds].sum(1)).flatten())
    window = 4
    sorted_read_counts = pd.Series(log10(read_counts.sort_values(ascending=False).values))
    x = log10(sorted_read_counts.groupby(sorted_read_counts).size()[::-1].cumsum())
    y = pd.Series(sorted_read_counts.groupby(sorted_read_counts).size()[::-1].index)
    threshold = int((pd.Series(pd.rolling_mean(y.diff().values/x.diff().values,window)).idxmin()-window/2.))
    read_threshold = read_counts.sort_values(ascending=False)[threshold]
    median_umis = read_counts.sort_values(ascending=False)[:threshold].median()
    fig = figure(figsize=(3,3))
    ax = fig.add_subplot(111)
    ax.plot(range(len(read_counts)),(read_counts.sort_values(ascending=False)).values,color='lightgray',linewidth=2)
    ax.plot(range(threshold),(read_counts.sort_values(ascending=False)).values[:threshold],color='g',linewidth=0,marker='o')
    ax.set_xscale('log')
    ax.set_yscale('log')
    _ = ax.set_xlabel('# Barcodes (logscale)')
    _ = ax.set_ylabel('# UMIs (logscale)')
    ax.text(1,10,' n_cells: %d\n read cutoff: %d\n median_umis: %d' %(threshold,read_threshold,median_umis))

In [ ]:
files=['/net/shendure/vol10/projects/BCBL/Mixing/results/20180419.AICS/s%d/read_assignment_s%d.csv' %(i,i) for i in [1,2,3]]
digital_count_matrix,all_genes,barcodes = load_data_combine_hex_combine_genes(files,
                                                                              read_cutoff=2000)

/net/shendure/vol10/projects/BCBL/Mixing/results/20180419.AICS/s1/read_assignment_s1.csv
/net/shendure/vol10/projects/BCBL/Mixing/results/20180419.AICS/s2/read_assignment_s2.csv
/net/shendure/vol10/projects/BCBL/Mixing/results/20180419.AICS/s3/read_assignment_s3.csv


In [ ]:
read_counts = pd.Series(np.array(digital_count_matrix.sum(1)).flatten())

In [ ]:
#samples_list = arange(48).reshape(4,12).astype(str)
#samples_list[:,:6] = 'donor'
#samples_list[:,6:8] = 'infusion'
#samples_list[:,8:] = 'pbmc'
#samples_list = samples_list.flatten()
#sample_dict = dict(zip(range(48),samples_list))

bcs = pd.Series(barcodes)
wellid = bcs.apply(lambda s:int(s.split('_')[1]))
#sample_types = wellid.apply(lambda x:sample_dict[x])

In [ ]:
plot_read_thresh()

In [ ]:
read_thresh = 2000
inds = find(read_counts>read_thresh)
digital_count_matrix = digital_count_matrix[inds]
barcodes = barcodes[inds]

In [ ]:
pd.Series(np.array((digital_count_matrix).sum(1)).flatten()).median()

In [ ]:
gene_counts = pd.Series(np.array((digital_count_matrix>0).sum(1)).flatten())

In [ ]:
gene_counts.median()

In [ ]:
gene_counts.hist(bins=40)

In [ ]:
obs = pd.DataFrame()
obs['sample'] = wellid[inds].values
X = digital_count_matrix.todense()
nonzero_gene_inds = find(X.sum(0)>0)
genes = all_genes.iloc[nonzero_gene_inds]
X = np.array(X[:,nonzero_gene_inds])
var = pd.DataFrame(index=genes.values)
adata = ad.AnnData(X, obs=obs, var=var, dtype='float64')

In [ ]:
sc.pp.filter_cells(adata, min_genes=100)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
mito_genes = [name for name in adata.var_names if (name.startswith('MT-') or name.startswith('mt-'))]
# for each cell compute fraction of counts in mito genes vs. all genes
# the ".A1" is only necessary, as X is sparse - it transform to a dense array after summing
adata.obs['percent_mito'] = np.sum(
    adata[:, mito_genes].X, axis=1) / np.sum(adata.X, axis=1)
# add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = np.sum(adata.X, axis=1)

In [ ]:
sc.pl.violin(adata, ['n_genes', 'n_counts', 'percent_mito'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='n_counts', y='percent_mito')
sc.pl.scatter(adata, x='n_counts', y='n_genes')

In [ ]:
adata_raw = sc.pp.log1p(adata, copy=True)
adata.raw = adata_raw

In [ ]:
sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
filter_result = sc.pp.filter_genes_dispersion(
    adata.X, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.filter_genes_dispersion(filter_result)

In [ ]:
adata = adata[:, filter_result.gene_subset]

In [ ]:
sc.pp.log1p(adata)

In [ ]:
#sc.pp.regress_out(adata, ['n_counts', 'percent_mito'])

In [ ]:
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.tl.pca(adata)

In [ ]:
adata.obsm['X_pca'] *= -1  # multiply by -1 to match Seurat R
sc.pl.pca_scatter(adata, color='MKI67_HUMAN', right_margin=0.2)

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
sc.tl.tsne(adata, n_pcs=30, random_state=2, use_fast_tsne=False)

In [ ]:
sc.pl.set_rcParams_Scanpy()

In [ ]:
#all_genes[all_genes.str.contains('MT-')]

In [ ]:
adata

In [ ]:
ax = sc.pl.tsne(adata, color=['n_counts','n_genes','percent_mito','MKI67_HUMAN'],
                color_map='Purples',alpha=0.5,size=30)

In [ ]:
sc.tl.louvain(adata, n_neighbors=10, resolution=0.75, recompute_graph=True)

In [ ]:
sc.pl.tsne(adata, color='louvain_groups',size=15)

In [ ]:
sc.pl.tsne(adata, color='sample',size=15)

In [ ]:
sc.tl.rank_genes_groups(adata, 'louvain_groups')
sc.pl.rank_genes_groups(adata, n_genes=20)

In [43]:
def get_cluster_exp(normed_counts,labels):
    cluster_names = pd.Series(unique(labels))
    cluster_names = cluster_names[cluster_names!=-1].values
    num_clust = len(cluster_names)
    cluster_fraction = {}
    cluster_counts = {}
    for i in cluster_names:
        if len(find(labels==i))>0:
            cur_matrix = normed_counts[find(labels==i)]
            cluster_fraction[i] = pd.Series(index=all_genes,data=np.array((cur_matrix>0).mean(0)).flatten())
            cluster_counts[i] = pd.Series(index=all_genes,data=np.array(cur_matrix.sum(0)).flatten())
    cluster_fraction = pd.DataFrame(cluster_fraction)
    cluster_counts = pd.DataFrame(cluster_counts)

    cluster_tpm = cluster_counts/cluster_counts.sum(0)*1e6+1

    cluster_diff_expression = {}
    for cur_cluster in cluster_counts.columns:
        cur_cluster_tpm = cluster_counts[cur_cluster]/cluster_counts[cur_cluster].sum()*1e6+1
        other_cluster_counts = cluster_counts.sum(1)-cluster_counts[cur_cluster]
        other_cluster_tpm = other_cluster_counts/other_cluster_counts.sum()*1e6+1
        cur_cluster_diff = log2(cur_cluster_tpm/other_cluster_tpm)
        cluster_diff_expression[cur_cluster]  = cur_cluster_diff
    cluster_diff_expression = pd.DataFrame(cluster_diff_expression)
    return cluster_fraction,cluster_tpm,cluster_diff_expression

def tsne_plot(X_tsne,labels,cluster_diff_expression,cluster_fraction,dotsize=0.1,num_genes=3,label_clusters=True):
    cluster_names = pd.Series(unique(labels))
    cluster_names = cluster_names[cluster_names!=-1].values
    num_clust = len(cluster_names)
    fig = figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    colors = sb.color_palette("hls", num_clust)
    c=0
    for i in cluster_names:
        if sum(labels==i)>0:
            cur_diff_gene = list(cluster_diff_expression[cluster_fraction[i]>0.2].sort_values(by=i,ascending=False).head(10).index[0:5])
            scatter(X_tsne[labels==i,0][:20000],
                    X_tsne[labels==i,1][:20000],color=colors[c],s=dotsize)
            if label_clusters:
                ax.text(X_tsne[labels==i,0].mean(),
                        X_tsne[labels==i,1].mean(),str(i),fontsize=16)
            c+=1
    c=0
    for i in cluster_names:
        if sum(labels==i)>0:
            cur_diff_gene = list(cluster_diff_expression[cluster_fraction[i]>0.2].sort_values(by=i,ascending=False).head(10).index[0:num_genes])
            cur_diff_gene = [s.split('_')[0] for s in cur_diff_gene]
            scatter([],
                    [],color=colors[c],s=70,label='Cluster_%s: '%str(i).split('.')[0]+str(cur_diff_gene)[1:-1]+', N='+str(sum(labels==i)))
            c+=1

    ax.yaxis.tick_left()
    ax.xaxis.tick_bottom()
    ax.set_xlabel('t-SNE 1',fontsize=fsize)
    ax.set_ylabel('t-SNE 2',fontsize=fsize)
    ax.legend(bbox_to_anchor=(2.25,1),fontsize=16)
    ax.axis([X_tsne[:,0].min()-2,X_tsne[:,0].max()+2,X_tsne[:,1].min()-2,X_tsne[:,1].max()+2])

In [239]:
all_genes = genes.iloc[human_genes]

In [241]:
normed_counts = normalize(X[:,human_genes],'l1',axis=1)

In [242]:
normed_counts.shape,labels.shape

((5631, 25658), (5631,))

In [243]:
labels = adata.obs['louvain_groups'].values
cluster_fraction,cluster_tpm,cluster_diff_expression = get_cluster_exp(normed_counts,labels)

In [244]:
cluster_fraction.to_csv('/net/shendure/vol10/projects/BCBL/Mixing/results/20180228.HumanCerebellum/fraction_cells_expressing.csv')
cluster_tpm.to_csv('/net/shendure/vol10/projects/BCBL/Mixing/results/20180228.HumanCerebellum/cluster_tpm.csv')
cluster_diff_expression.to_csv('/net/shendure/vol10/projects/BCBL/Mixing/results/20180228.HumanCerebellum/cluster_log2_diff.csv')

In [247]:
diff_genes = {}
for i in range(21):
    diff_genes[i] = list(cluster_diff_expression[cluster_fraction[str(i)]>0.2].sort_values(by=str(i),ascending=False)[:10].index)
diff_genes = pd.DataFrame(diff_genes)

In [249]:
diff_genes.T

,0,1,2,3,4,5,6,7,8,9
0,TRHDE,AC068490.2,FSTL4,RNF220,NTN1,PLXNB2,RYR3,ROR1,ST18,CNTN5
1,CTD-3088G3.8,FREM2,CCDC102B,MEGF10,RGS20,TNC,CTNNA3,FREM1,PRTG,PAX3
2,GRIN3A,KIAA1239,GRIN1,ANKRD30BL,CHD5,SCN9A,MCF2L2,ZNF804B,RP11-307P5.1,KSR2
3,SATB2,MPPED1,MIR137HG,CCBE1,SLA,PLXNA4,DOK5,KCNK2,MEF2C,SLC44A5
4,CA8,ITPR1,PDE11A,COL24A1,IL1RAPL2,ESRRB,SEMA3D,PCP4,PTGER3,PRMT8
5,HSPD1,RPS6,PDE3A,HSP90AB1,HSP90AA1,PTMA,MT-ND4,BASP1,SNORA79,PRRC2C
6,UNC5D,RP11-436D23.1,SEZ6,ENC1,CLMP,LRP8,KCNN3,SLA,KCNQ3,EPHA3
7,FGF5,LINC01036,FAT2,CTC-340A15.2,NTF3,CTC-535M15.2,SYT9,GNA14,AC011306.2,KCNAB1
8,RP11-76I14.1,TOP2A,LTBP1,PRDM16,GLI3,MKI67,CELSR1,CENPE,APOLD1,CENPF
9,AQP4,RP11-561I11.4,SPARCL1,RNF219-AS1,EDNRB,SLC6A11,BCAN,MMD2,EEPD1,LRIG1


In [259]:
X.shape

(5631, 35242)